In [ ]:
import duckdb

# Caminhos para os bancos de dados
db1_path = 'fineweb.duckdb'  # Substitua pelo caminho do seu primeiro banco de dados
db2_path = 'books.duckdb'  # Substitua pelo caminho do seu segundo banco de dados

# Alias para o segundo banco de dados
alias_db2 = 'db2_alias'

# Conectar ao banco de dados principal
conn = duckdb.connect(database=db1_path, read_only=False)

# Anexar o segundo banco de dados
conn.execute(f"ATTACH DATABASE '{db2_path}' AS {alias_db2}")

# Verificar esquemas (opcional, mas recomendado)
schema_main = conn.execute("DESCRIBE dataset").fetchdf()
schema_db2 = conn.execute(f"DESCRIBE {alias_db2}.dataset").fetchdf()

if not schema_main.equals(schema_db2):
	raise Exception("Os esquemas das tabelas 'dataset' nos dois bancos de dados não são compatíveis.")

# Executar a consulta modificada
df_training = conn.execute(f"""
WITH combined_dataset AS (
    SELECT * FROM dataset
    UNION ALL
    SELECT * FROM {alias_db2}.dataset
),
sampled_a AS (
    SELECT id, indice, content, name
    FROM (
        SELECT
            id,
            indice,
            content,
            name,
            ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
        FROM combined_dataset
    ) sub
    WHERE rn <= 1
),
sampled_b AS (
    SELECT id, indice, content, name
    FROM (
        SELECT
            id,
            indice,
            content,
            name,
            ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
        FROM combined_dataset
    ) sub
    WHERE rn <= 1
)
SELECT
    a.name AS name,
    a.content AS content1,
    b.content AS content2,
    -- Cálculo original do target_transformado
    SIGN(a.indice - b.indice) * LN(1 + ABS(a.indice - b.indice)) AS target_transformed,
    -- Aplicação da função sigmoide no target_transformado
    1 / (1 + EXP(- (SIGN(a.indice - b.indice) * LN(1 + ABS(a.indice - b.indice))))) AS target_transformed_sigmoid
FROM sampled_a a
JOIN sampled_b b
    ON a.name = b.name
ORDER BY RANDOM()
LIMIT 50000;
""").df()

conn.close()

df_training

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_training['target_transformed'], bins=50, alpha=0.5, label='target_transformed')
plt.legend()
plt.show()

In [7]:
# Cria um iterador a partir do dataset
train_iter = iter(training_dataset)

# Obtém a próxima amostra
embeddings, lengths, target = next(train_iter)

# Exibe a amostra
print("Embeddings:", embeddings.shape)
print("Lengths:", lengths)
print("Target:", target)

Embeddings: torch.Size([2, 7, 2048])
Lengths: tensor([7, 7])
Target: tensor(5.9648, device='cuda:0', dtype=torch.float16)


In [1]:
import torch
import gc

if not torch.cuda.is_available():
    print("CUDA is not available. Please ensure you have a compatible GPU and drivers installed.")
else:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")

Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer
import os

# Definir paralelismo corretamente
os.environ["TOKENIZERS_PARALLELISM"] = "true"

class TextToEmbedding:
    def __init__(self, weights_path, num_ids=128256, vector_size=2048, device='cpu'):
        """
        Inicializa a classe TextToEmbedding.

        Args:
            weights_path (str): Caminho para o arquivo .npy que contém os pesos.
            num_ids (int, opcional): Número total de IDs. Padrão é 128256.
            vector_size (int, opcional): Tamanho de cada vetor de embedding. Padrão é 2048.
            device (str, opcional): Dispositivo para carregar os tensores ('cpu' ou 'cuda'). Padrão é 'cpu'.
        """
        self.device = device

        # Carrega o tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B", use_fast=True)
        
        # Carrega os pesos a partir do arquivo .npy
        try:
            weights_np = np.load(weights_path)
            self.weights = torch.from_numpy(weights_np).to(self.device)
        except FileNotFoundError:
            raise FileNotFoundError(f"O arquivo de pesos '{weights_path}' não foi encontrado.")
        except Exception as e:
            raise RuntimeError(f"Erro ao carregar os pesos: {e}")
        
        # Verifica a forma dos pesos
        if self.weights.shape != (num_ids, vector_size):
            raise ValueError(f"O formato do arquivo weights.npy é {self.weights.shape}, mas era esperado {(num_ids, vector_size)}.")

    def text_to_embedding(self, input_texts):
        """
        Converte uma lista de textos de entrada em seus embeddings correspondentes e retorna os comprimentos reais.

        Args:
            input_texts (List[str]): Lista de textos a serem convertidos em embeddings.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: 
                - Tensor contendo os embeddings correspondentes aos IDs dos textos.
                  Forma: (batch_size, sequence_length, vector_size)
                - Tensor contendo os comprimentos reais de cada sequência no batch.
                  Forma: (batch_size,)
        """
        if not isinstance(input_texts, list):
            raise TypeError("input_texts deve ser uma lista de strings.")
        
        # Tokeniza os textos de entrada com padding e truncamento
        encoding = self.tokenizer(
            input_texts,
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        input_ids = encoding.input_ids.to(self.device)
        attention_mask = encoding.attention_mask.to(self.device)
        
        # Assegura que os IDs estão no tipo correto para indexação
        ids_tensor = input_ids.type(torch.long)
        
        # Obtém os embeddings correspondentes aos IDs
        # Forma resultante: (batch_size, sequence_length, vector_size)
        embedding = self.weights[ids_tensor]
        
        # Calcula os comprimentos reais de cada sequência
        # Somando os valores da máscara de atenção, que são 1 para tokens reais e 0 para padding
        lengths = attention_mask.sum(dim=1)
        
        return embedding, lengths

# Exemplo de uso
embedding_generator = TextToEmbedding("weights_half.npy", device='cpu')

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class SequenceDataset_val(Dataset):
	def __init__(self, df):
		self.df = df.reset_index(drop=True)

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		row = self.df.iloc[idx]
		embeddings, lengths = embedding_generator.text_to_embedding([row['content1'],row['content1']])

		target = torch.tensor(row['target_transformed'], dtype=torch.float16)

		return embeddings,lengths, target

In [3]:
import torch
from torch.utils.data import IterableDataset, DataLoader
import duckdb
from tqdm import tqdm
import pandas as pd

# Paths to the databases
db1_path = 'fineweb.duckdb'
db2_path = 'books.duckdb'

class IterableSequenceDataset(IterableDataset):
	def __init__(self, db1_path, db2_path, batch_size=100):
		super(IterableSequenceDataset, self).__init__()
		self.db1_path = db1_path
		self.db2_path = db2_path
		self.batch_size = batch_size

	def _fetch_data(self, conn):
		query = f"""
			WITH combined_dataset AS (
				SELECT * FROM main.dataset
				UNION ALL
				SELECT * FROM db2.dataset
			),
			sampled_a AS (
				SELECT id, indice, content, name
				FROM (
					SELECT
						id,
						indice,
						content,
						name,
						ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
					FROM combined_dataset
				) sub
				WHERE rn <= 1
			),
			sampled_b AS (
				SELECT id, indice, content, name
				FROM (
					SELECT
						id,
						indice,
						content,
						name,
						ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
					FROM combined_dataset
				) sub
				WHERE rn <= 1
			)
			SELECT
				a.name AS name,
				a.content AS content1,
				b.content AS content2,
				SIGN(a.indice - b.indice) * LN(1 + ABS(a.indice - b.indice)) AS target_transformed
			FROM sampled_a a
			JOIN sampled_b b
				ON a.name = b.name
			ORDER BY RANDOM()
			LIMIT {self.batch_size};
		"""
		return conn.execute(query).df()

	def __iter__(self):
		"""
		Iterator that generates data continuously.

		Yields:
			tuple: (embeddings, lengths, target) for each sample.
		"""
		# Establish connection with DuckDB
		conn = duckdb.connect(database=self.db1_path, read_only=True)
		conn.execute("SET enable_progress_bar=false")
		
		# Attach the second database with an alias 'db2'
		conn.execute(f"ATTACH '{self.db2_path}' AS db2")

		try:
			while True:
				df = self._fetch_data(conn)
				for _, row in df.iterrows():
					embeddings, lengths = embedding_generator.text_to_embedding([row['content1'], row['content2']])
					embeddings = embeddings.half().to('cuda')

					# Manter lengths como CPU int64 tensor
					lengths = lengths.to(torch.int64).to('cpu')

					target = torch.tensor(row['target_transformed'], dtype=torch.float16).to('cuda')

					yield embeddings, lengths, target
		finally:
			# Ensure the connection is closed when the iterator is finalized
			conn.close()

In [4]:
import numpy as np

def signed_log_transform(y):
    return np.sign(y) * np.log1p(np.abs(y))

def inverse_signed_log_transform(y_transformed):
    return np.sign(y_transformed) * (np.expm1(np.abs(y_transformed)))

def dissimilaridade(S):
	epsilon = 0
	if S == 0:
		max_x = np.log(np.finfo(np.float32).max)
		epsilon = (1/(max_x-1))
        
	return (1 - (S+epsilon)) / (S+epsilon)

In [5]:
import pandas as pd

data = {
    'target_transformed': [1]*50,
    "content1": ["A tempestade durou várias horas durante a noite.", "Ele estudou intensamente para o exame.", "A fábrica reduziu a emissão de poluentes.", "Maria começou a praticar exercícios regularmente.", "O motor do carro parou de funcionar.", "Houve uma forte seca na região.", "Ela começou a dormir melhor.", "A escola adotou uma alimentação saudável.", "O projeto teve apoio governamental.", "A internet caiu durante a reunião.", "Ele começou a economizar dinheiro mensalmente.", "A estrada estava em péssimo estado de conservação.", "O sistema de ar condicionado foi desligado no escritório.", "A empresa investiu em marketing digital.", "Ana não revisou o relatório antes de enviar.", "As crianças brincaram no parque até tarde.", "O curso de capacitação foi oferecido aos funcionários.", "Ele comprou um celular novo com câmera de alta qualidade.", "A reforma no prédio foi concluída.", "A região teve um crescimento populacional rápido.", "Carla começou a meditar diariamente.", "O atleta intensificou seu treino antes da competição.", "O sinal de celular na região foi ampliado.", "As temperaturas caíram drasticamente durante o inverno.", "Ele não configurou o alarme antes de dormir.", "A empresa ofereceu benefícios extras para seus funcionários.", "Houve um vazamento de gás na cozinha do restaurante.", "A biblioteca da escola foi modernizada com novos livros e tecnologia.", "Marcos passou a fazer pausas regulares durante o expediente.", "A empresa desenvolveu um aplicativo intuitivo para clientes.", "Devido ao aumento das chuvas nas últimas semanas, o nível dos rios subiu rapidamente e ultrapassou a capacidade das barragens.", "Ela decidiu começar uma rotina de alimentação balanceada, exercícios físicos regulares e meditação diária para melhorar sua saúde física e mental.", "A empresa, que sofria com baixos índices de produtividade, implementou uma nova estratégia de gestão focada no desenvolvimento dos funcionários e na cultura organizacional.", "O governo lançou um programa nacional de reciclagem e incentivou a participação ativa dos cidadãos por meio de campanhas de conscientização em escolas, empresas e residências.", "Após uma longa estiagem que afetou grande parte do território agrícola do país, o governo implementou um pacote emergencial de apoio aos agricultores, incluindo subsídios e incentivos para a recuperação das lavouras.", "Ele deixou o celular carregando a noite inteira sem usar carregadores de segurança e em um local sem ventilação.", "A comunidade local se uniu para limpar e revitalizar a praça abandonada, que estava sem manutenção há anos e havia se tornado um ponto de descarte irregular de lixo.", "Durante a reforma do prédio, descobriu-se que a estrutura tinha falhas graves, e o prazo para conclusão foi ampliado em seis meses para garantir a segurança.", "Um furacão atingiu a região costeira do país, com ventos de mais de 200 km/h, causando destruição em várias cidades e deixando milhares de pessoas desabrigadas.", "Ela esqueceu de regar a planta de sua varanda durante o mês todo, e o clima estava seco.", "João quebrou o braço jogando futebol.", "Devido a uma combinação de fatores climáticos extremos, incluindo ventos fortes e chuvas intensas, a região sofreu graves danos estruturais e interrupção no fornecimento de energia elétrica por vários dias.", "Ela esqueceu de levar o guarda-chuva.", "A empresa adotou práticas sustentáveis em sua produção, reduzindo o uso de plástico e implementando programas de reciclagem.", "A criança se recusou a comer legumes durante toda a semana.", "Após anos de pesquisa e desenvolvimento, a equipe científica finalmente descobriu um método eficiente para produzir energia limpa a partir de fontes renováveis.", "O café da manhã foi esquecido.", "A iniciativa comunitária organizou mutirões de limpeza nas praias locais regularmente durante o verão.", "Ele atualizou seu currículo e participou de várias entrevistas de emprego nos últimos meses.", "O artista decidiu experimentar novas técnicas em suas pinturas, incorporando elementos digitais e materiais reciclados em suas obras."],
    "content2": ["Pela manhã, muitas ruas estavam alagadas.", "Conseguiu uma nota alta na prova.", "A qualidade do ar na cidade melhorou.", "Ela perdeu peso e aumentou sua energia.", "Ele teve que chamar o guincho para levar o carro à oficina.", "As plantações foram prejudicadas, e a colheita foi menor.", "Sua disposição durante o dia melhorou significativamente.", "Os alunos passaram a ter mais energia e melhor concentração.", "Conseguiu concluir as fases iniciais rapidamente.", "A comunicação com a equipe foi interrompida.", "Conseguiu juntar uma quantia para uma viagem.", "O trânsito ficou mais lento e perigoso para os motoristas.", "O ambiente ficou quente e desconfortável para os funcionários.", "As vendas aumentaram significativamente.", "O documento continha erros e precisou ser corrigido.", "Elas ficaram cansadas e dormiram rapidamente ao chegar em casa.", "Eles melhoraram suas habilidades e eficiência no trabalho.", "Passou a tirar fotos mais nítidas e de melhor resolução.", "O local ficou mais seguro e esteticamente agradável.", "A demanda por moradias e serviços aumentou consideravelmente.", "Ela sentiu uma melhora no seu foco e redução do estresse.", "Teve um melhor desempenho e conquistou o primeiro lugar.", "A conexão ficou mais estável e acessível para os moradores.", "A procura por agasalhos e cobertores aumentou nas lojas.", "Acabou se atrasando para o trabalho na manhã seguinte.", "A satisfação e motivação dos colaboradores aumentaram.", "O local foi evacuado por segurança, e o serviço foi interrompido temporariamente.", "Os alunos começaram a frequentá-la mais e a melhorar seu desempenho acadêmico.", "Ele se sentiu mais produtivo e menos cansado ao final do dia.", "O número de usuários aumentou rapidamente.", "Diversas áreas urbanas e rurais foram afetadas por inundações, forçando muitas famílias a deixarem suas casas temporariamente.", "Em poucos meses, notou uma grande melhora em sua disposição, concentração e níveis de energia, além de perder peso.", "Em menos de um ano, a moral da equipe melhorou, a rotatividade diminuiu, e a produtividade geral aumentou em cerca de 30%.", "Como resultado, houve uma redução significativa na quantidade de resíduos sólidos em aterros e uma maior economia de recursos naturais.", "Em um ano, a produção agrícola voltou a níveis estáveis, e o impacto econômico negativo foi mitigado, beneficiando a população.", "Pela manhã, o dispositivo estava superaquecido e apresentou danos permanentes na bateria, reduzindo sua capacidade de funcionamento.", "Com a revitalização, a praça voltou a ser um local de encontro para moradores, e a segurança da área também melhorou.", "Apesar do atraso, a reforma resultou em um edifício mais seguro, confortável e com um aumento significativo no valor do imóvel.", "A resposta emergencial foi mobilizada rapidamente, com abrigos temporários e ajuda humanitária distribuída para minimizar o impacto nas vítimas.", "A planta murchou completamente e, infelizmente, não conseguiu ser recuperada, tendo que ser substituída.", "Ele ficou impossibilitado de trabalhar por duas semanas.", "A comunidade teve que recorrer a abrigos temporários, e a economia local sofreu uma queda significativa devido à paralisação das atividades comerciais.", "Ficou molhada durante o trajeto para o trabalho.", "A reputação da empresa melhorou, atraindo consumidores conscientes e aumentando as vendas em 20%.", "Sua mãe ficou preocupada com a falta de nutrientes na alimentação dele e decidiu consultar um nutricionista.", "Isso resultou na redução significativa das emissões de carbono e no avanço tecnológico sustentável, beneficiando o meio ambiente globalmente.", "Ela sentiu fome e falta de energia durante a manhã de trabalho.", "As praias ficaram mais limpas e atraentes para turistas, além de promover a conscientização ambiental entre os moradores.", "Finalmente recebeu uma oferta de trabalho em uma empresa renomada, melhorando sua estabilidade financeira.", "Suas peças ganharam destaque em exposições internacionais, ampliando seu reconhecimento e alcance no mercado de arte."]
}

# Crie o DataFrame
df_validation = pd.DataFrame(data)

# Adicionar frases iguais
df_zero = pd.DataFrame({
    'target_transformed': [0] * len(data['target_transformed']),
    'content1': df_validation['content1'],
    'content2': df_validation['content1']
})

# Adicionar frases com a ordem contrária
df_reversed = pd.DataFrame({
    'target_transformed': [-1] * len(data['target_transformed']),
    'content1': df_validation['content2'],
    'content2': df_validation['content1']
})

# Concatenar os dois DataFrames
df_validation = pd.concat([df_zero, df_validation, df_reversed], ignore_index=True)

df_validation

,target_transformed,content1,content2
0,0,A tempestade durou várias horas durante a noite.,A tempestade durou várias horas durante a noite.
1,0,Ele estudou intensamente para o exame.,Ele estudou intensamente para o exame.
2,0,A fábrica reduziu a emissão de poluentes.,A fábrica reduziu a emissão de poluentes.
3,0,Maria começou a praticar exercícios regularmente.,Maria começou a praticar exercícios regularmente.
4,0,O motor do carro parou de funcionar.,O motor do carro parou de funcionar.
...,...,...,...
145,-1,Isso resultou na redução significativa das emi...,"Após anos de pesquisa e desenvolvimento, a equ..."
146,-1,Ela sentiu fome e falta de energia durante a m...,O café da manhã foi esquecido.
147,-1,As praias ficaram mais limpas e atraentes para...,A iniciativa comunitária organizou mutirões de...
148,-1,Finalmente recebeu uma oferta de trabalho em u...,Ele atualizou seu currículo e participou de vá...


In [ ]:
import torch.nn as nn

def collate_fn(batch):
    embeddings_list, lengths_list, targets = zip(*batch)
        
    # Empilhar embeddings: forma resultante (batch_size, 2, seq_len, vector_size)
    embeddings = torch.stack(embeddings_list)
    
    # Empilhar lengths: forma resultante (batch_size, 2)
    lengths = torch.stack(lengths_list)
    targets = torch.stack(targets)
    
    return embeddings, lengths, targets

In [7]:
batch_size = 100

training_dataset = IterableSequenceDataset(db1_path, db2_path)
train_dataloader = DataLoader(
  training_dataset, 
  batch_size=batch_size, 
  num_workers=1,
  collate_fn=collate_fn,
  persistent_workers=True
)

validation_dataset = SequenceDataset_val(df_validation)
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=150,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
    collate_fn=collate_fn
)

In [ ]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTMSequenceToOneModel(nn.Module):
    def __init__(self, input_size=2048, hidden_size=256, num_layers=3):
        super(LSTMSequenceToOneModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM bidirecional
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.1
        )

        # Tamanho final após bidirecional
        final_hidden_size = hidden_size * 2  # bidirectional

        # Camadas totalmente conectadas
        self.fc1 = nn.Linear(final_hidden_size, 64)
        self.leaky_relu = nn.LeakyReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, sequences, lengths):
        """
        Forward pass do modelo.

        Args:
            sequences (Tensor): Tensor de embeddings de forma (batch, seq_len, input_size).
            lengths (Tensor): Tensor 1D CPU int64 contendo os comprimentos das sequências.

        Returns:
            Tensor: Saída do modelo.
        """
        # Verificações de dtype e dispositivo
        assert sequences.dtype == torch.float16, "Sequences devem ser float16"
        assert lengths.dtype == torch.int64 and lengths.device == torch.device('cpu'), "Lengths devem ser int64 no CPU"

        # Ordenar as sequências por comprimento decrescente
        lengths_sorted_cpu, perm_idx = lengths.sort(0, descending=True)
        sequences_sorted = sequences[perm_idx]

        # Empacotar as sequências
        packed_input = pack_padded_sequence(sequences_sorted, lengths_sorted_cpu, batch_first=True)

        # Passar pela LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_input)

        # Desempacotar
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)

        # Desordenar de volta para a ordem original
        _, unperm_idx = perm_idx.sort(0)
        lstm_out = lstm_out[unperm_idx]
        lengths_sorted_cpu = lengths_sorted_cpu[unperm_idx]

        # Converter lengths para float16 e mover para GPU para cálculos
        lengths_sorted_gpu = lengths_sorted_cpu.to(dtype=torch.float16).to(lstm_out.device)

        # Criar máscara na GPU
        mask = torch.arange(lstm_out.size(1), device=lstm_out.device).unsqueeze(0) < lengths_sorted_gpu.unsqueeze(1)
        mask = mask.unsqueeze(2).to(lstm_out.dtype)  # Converter para float16

        # Aplicar máscara
        lstm_out = lstm_out * mask

        # Calcular a média ponderada
        lstm_out_mean = lstm_out.sum(1) / lengths_sorted_gpu.unsqueeze(1)

        # Passar pelas camadas totalmente conectadas
        out = self.fc1(lstm_out_mean)
        out = self.leaky_relu(out)
        out = self.fc2(out)
        return out[0][0]

In [22]:
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn as nn

class LSTMSequenceToOneLightning(pl.LightningModule):
    def __init__(self, input_size=2048, hidden_size=256, num_layers=3, lr=1e-3):
        super(LSTMSequenceToOneLightning, self).__init__()
        self.save_hyperparameters()

        # Instanciar o modelo base
        self.model = LSTMSequenceToOneModel(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers
        )

        # Função de perda
        self.criterion = nn.MSELoss()

    def forward(self, sequence, length):
        return self.model(sequence, length)

    def training_step(self, batch, batch_idx):
        # Como batch_size=1, desempacotamos diretamente
        sequences, lengths, targets = batch  # Cada tensor tem shape (1, ...)
        print(sequences.shape)
        # Remover a dimensão do batch
        sequence = sequences.squeeze(0)  # Shape: (sequence_length, input_size)
        length = lengths.squeeze(0).to(torch.int64).cpu()  # Shape: ()
        target = targets.squeeze(0)  # Shape: (output_size)

        # Forward pass
        output = self(sequence, length)

        # Computar a perda
        loss = self.criterion(output, target)

        # Logar a perda de treinamento
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        sequences, lengths, targets = batch
        loss = 0
        batch_size = len(sequences)
        for i in range(batch_size):
            seq = sequences[i]
            length = lengths[i].to(torch.int64).to('cpu')
            target = targets[i]
            output = self(seq, length)
            loss += self.criterion(output, target)
        loss = loss / batch_size
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True,logger=True)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# Instanciar o modelo LightningModule
model = LSTMSequenceToOneLightning()

# Definir callbacks, por exemplo, ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)

# Instanciar o Trainer
trainer = Trainer(
    max_epochs=10,
    precision=16,                            # Usa precisão de 16 bits se desejado
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices='auto',
    callbacks=[checkpoint_callback],
    accumulate_grad_batches=1,               # Ajuste se usar acumulação de gradientes
    log_every_n_steps=50,                    # Ajuste a frequência de log
    deterministic=True                       # Para reprodutibilidade
)

# Iniciar o treinamento
trainer.fit(
    model,
    train_dataloaders=train_dataloader, #IterableDataset
    val_dataloaders=validation_dataloader #Dataset
)

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | model     | LSTMSequenceToOneModel | 7.9 M  | train
1 | criterion | MSELoss                | 0      | train
-------------------------------------------------------------
7.9 M     Trainable params
0         Non-trainable params
7.9 M     Total params
31.638    Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# Verificar se CUDA está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instanciar o modelo Lightning
model = LSTMSequenceToOneLightning()

# Definir callbacks, se necessário (opcional)
checkpoint_callback = ModelCheckpoint(
	monitor='val_loss',
	dirpath='checkpoints',
	filename='best-checkpoint',
	save_top_k=1,
	mode='min'
)

# Instanciar o Trainer
trainer = Trainer(
	max_epochs=10,
  	precision=16,
	accelerator='gpu' if torch.cuda.is_available() else 'cpu',
	devices=1 if torch.cuda.is_available() else None,
	callbacks=[checkpoint_callback]
)

# Iniciar o treinamento
trainer.fit(model, training_dataset, validation_dataloader)

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | model     | LSTMSequenceToOneModel | 7.9 M  | train
1 | criterion | MSELoss                | 0      | train
-------------------------------------------------------------
7.9 M     Trainable params
0         Non-trainable params
7.9 M     Total params
31.638    Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training: |          | 0/? [00:00<?, ?it/s]

IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [ ]:
# Initialize the Lightning module
model = SequenceToOneLightningModule(
    input_size=2048,
    hidden_size=256,
    num_layers=3,
    learning_rate=1e-3
)

# Start training
trainer.fit(model, train_dataloaders=training_dataset, val_dataloaders=validation_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | model     | LSTMSequenceToOneModel | 7.9 M  | train
1 | criterion | HuberLoss              | 0      | train
-------------------------------------------------------------
7.9 M     Trainable params
0         Non-trainable params
7.9 M     Total params
31.638    Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'tuple' object has no attribute 'dtype'

In [ ]:
# Cria um iterador a partir do dataset
train_iter = iter(training_dataset)

# Obtém a próxima amostra
embeddings, lengths, target = next(train_iter)

# Instanciar o modelo base
core_model = LSTMSequenceToOneLightning().to('cuda').half()

# Passar os dados para GPU e converter para half precision
embeddings = embeddings.to('cuda').half()
lengths = lengths

# Executar o forward
outputs = core_model(embeddings, lengths)
outputs

tensor([-0.0413, -0.0410], device='cuda:0', dtype=torch.float16,
       grad_fn=<SqueezeBackward1>)

In [9]:
model_order = LSTMSequenceToOneModel().to('cuda').half()
#criterion = nn.MSELoss()
criterion = nn.HuberLoss()

In [10]:
# Cria um iterador a partir do dataset
train_iter = iter(training_dataset)

# Obtém a próxima amostra
embeddings, lengths, target = next(train_iter)

# Exibe a amostra
print("Embeddings:", embeddings.shape)
print("Lengths:", lengths)
print("Target:", target)

Embeddings: torch.Size([2, 98, 2048])
Lengths: tensor([54, 98])
Target: tensor(-6.9531, device='cuda:0', dtype=torch.float16)


In [10]:
optimizer = torch.optim.Adam(
    model_order.parameters(), lr=0.1, weight_decay=1e-6)

# Inicializar o scheduler ReduceLROnPlateau
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=1, verbose=True
)

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
# Inicializar o contador de passos
step = 0
steps_per_epoch = 10  # Número de batches por época
validation_steps = None  # Se você deseja limitar os passos de validação, defina um número inteiro

In [12]:
num_epochs = 100

In [13]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter('runs/treinamento_regressao_rnn')

# Wrap the epoch loop with tqdm for progress
epoch_progress = tqdm(range(num_epochs), desc='Treinamento', unit='época')

for epoch in epoch_progress:
    epoch_progress.set_description(f'Época {epoch+1}/{num_epochs}')

    # Training phase
    model_order.train()
    train_loss = 0.0

    for batch in tqdm(train_dataloader, desc='Treinando', leave=False):
        embeddings_batch, lengths_batch, targets_batch = batch

        optimizer.zero_grad()

        # Forward pass
        outputs = model_order(embeddings_batch, lengths_batch)
        loss = criterion(outputs, targets_batch)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * targets_batch.size(0)

        # Log training loss
        writer.add_scalar('Perda_Treinamento_Passo', loss.item(), step)
        step += 1

    # Calculate average training loss
    avg_train_loss = train_loss / len(train_dataloader.dataset)

    # Validation phase
    model_order.eval()
    validation_loss = 0.0
    correct_sign = 0
    total = 0

    with torch.no_grad():
        for batch in validation_dataloader:
            embeddings_batch, lengths_batch, targets_batch = batch

            outputs = model_order(embeddings_batch, lengths_batch)
            loss = criterion(outputs, targets_batch)
            validation_loss += loss.item() * targets_batch.size(0)

            # Sign comparison logic
            matches = (
                ((targets_batch == -1) & (outputs < -1.71)) |  # Rule for -1
                ((targets_batch == 0) & ((-1.71 < outputs) & (outputs < 1.71))) |  # Rule for 0
                ((targets_batch == 1) & (1.71 < outputs))     # Rule for 1
            )

            correct_sign += matches.sum().item()
            total += targets_batch.size(0)

    sign_accuracy = correct_sign / total
    avg_validation_loss = validation_loss / total

    # Log validation metrics
    writer.add_scalar('Perda_Validação_Epoca', avg_validation_loss, epoch+1)
    writer.add_scalar('Acurácia_Sinal_Validação', sign_accuracy, epoch+1)

    # Update scheduler
    scheduler.step(avg_validation_loss)

    # Log learning rate
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Taxa_Aprendizado', current_lr, epoch+1)

    # Update progress bar
    epoch_progress.set_postfix({
        'Perda Treinamento': f'{avg_train_loss:.3f}',
        'Perda Validação': f'{avg_validation_loss:.3f}',
        'Acurácia Sinal': f'{sign_accuracy*100:.2f}%',
        'LR': f'{current_lr:.8f}'
    })

    # Clear GPU cache
    torch.cuda.empty_cache()

# Close the SummaryWriter
writer.close()

Época 1/100:   0%|          | 0/100 [00:00<?, ?época/s]

Época 1/100:   0%|          | 0/100 [00:17<?, ?época/s]


RuntimeError: stack expects each tensor to be equal size, but got [2, 6, 2048] at entry 0 and [2, 12, 2048] at entry 1

In [ ]:
# Inicializar o writer do TensorBoard
writer = SummaryWriter('runs/treinamento_regressao_rnn')

# Envolver o loop de épocas com tqdm para mostrar o progresso das épocas
epoch_progress = tqdm(range(num_epochs), desc='Treinamento', unit='época')

for epoch in epoch_progress:
	epoch_progress.set_description(f'Época {epoch+1}/{num_epochs}')

	# Treinamento
	model_order.train()
	train_loss = 0.0

	for embeddings_batch, lengths_batch, targets_batch in tqdm(train_dataloader, desc='Treinando', leave=False):
		optimizer.zero_grad()

		# Forward pass
		outputs = model_order(embeddings_batch, lengths_batch)
		loss = criterion(outputs, targets_batch)

		# Backward pass e otimização
		loss.backward()
		optimizer.step()

		train_loss += loss.item() * targets_batch.size(0)

		# Registrar a perda de treinamento a cada passo
		writer.add_scalar('Perda_Treinamento_Passo', loss.item(), step)
		step += 1

	# Calcular a perda média de treinamento para a época
	avg_train_loss = train_loss / len(train_dataloader.dataset)

	# Validação (mantém-se similar, mas verifique a passagem correta dos dados)
	model_order.eval()
	validation_loss = 0.0
	correct_sign = 0
	total = 0

	with torch.no_grad():
		for embeddings_batch, lengths_batch, targets_batch in validation_dataloader:

			outputs = model_order(embeddings_batch, lengths_batch)
			loss = criterion(outputs, targets_batch)
			validation_loss += loss.item() * targets_batch.size(0)

			# Lógica de comparação
			matches = (
				((targets_batch == -1) & (outputs < -1.71)) |  # Regra para -1
				((targets_batch == 0) & ((-1.71 < outputs) & (outputs < 1.71))) |  # Regra para 0
				((targets_batch == 1) & (1.71 < outputs))     # Regra para 1
			)

			correct_sign += matches.sum().item()
			total += targets_batch.size(0)

	sign_accuracy = correct_sign / total
	avg_validation_loss = validation_loss / total

	# Registrar as métricas de validação por época
	writer.add_scalar('Perda_Validação_Epoca', avg_validation_loss, epoch+1)
	writer.add_scalar('Acurácia_Sinal_Validação', sign_accuracy, epoch+1)

	# Passar a perda de validação para o scheduler
	scheduler.step(avg_validation_loss)

	# Registrar a taxa de aprendizado atual (opcional)
	current_lr = optimizer.param_groups[0]['lr']
	writer.add_scalar('Taxa_Aprendizado', current_lr, epoch+1)

	# Atualizar a descrição da barra com as perdas atuais e acurácia do sinal
	epoch_progress.set_postfix({
		'Perda Treinamento': f'{avg_train_loss:.3f}',
		'Perda Validação': f'{avg_validation_loss:.3f}',
		'Acurácia Sinal': f'{sign_accuracy*100:.2f}%',
		'LR': f'{current_lr:.8f}'
	})

	# Liberar cache da GPU após cada época
	torch.cuda.empty_cache()

# Fechar o SummaryWriter ao final do treinamento
writer.close()

In [ ]:
import os

# Diretório onde os modelos serão salvos
save_dir = './saved_fineweb2'
os.makedirs(save_dir, exist_ok=True)

# Salvando o estado do modelo
model_path = os.path.join(save_dir, 'regression_rnn_model.pth')
torch.save(model_order.state_dict(), model_path)
print(f"Modelo salvo em {model_path}")

# Opcional: Salvando o estado do otimizador
optimizer_path = os.path.join(save_dir, 'optimizer_state.pth')
torch.save(optimizer.state_dict(), optimizer_path)
print(f"Estado do otimizador salvo em {optimizer_path}")

# Opcional: Salvar a época atual para retomar o treinamento
epoch_path = os.path.join(save_dir, 'last_epoch.pth')
torch.save({'epoch': epoch, 'model_state_dict': model_order.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_validation_loss}, epoch_path)
print(f"Checkpoint salvo em {epoch_path}")

In [ ]:
import os
model_path = os.path.join('./saved_fineweb', 'regression_rnn_model.pth')

# Carregar o estado do modelo
model_order.load_state_dict(torch.load(model_path, map_location='cuda'))

# Colocar o modelo em modo de avaliação
#model_order.eval()

In [ ]:
model_order.eval()

sequences1_batch = embedding_generator.text_to_embedding_parallel(df_validation['content1'].tolist(), max_workers=20)
sequences2_batch = embedding_generator.text_to_embedding_parallel(df_validation['content2'].tolist(), max_workers=20)

sequences1_batch = [s.to('cuda') for s in sequences1_batch]
sequences2_batch = [s.to('cuda') for s in sequences2_batch]

targets_batch = targets_batch.to('cuda')

# Realizar a previsão
with torch.no_grad():
	output = model_order(sequences1_batch, sequences2_batch)

	matches = (
		((targets_batch == -1) & (outputs < -1.71)) |  # Regra para -1
		((targets_batch == 0) & ((-1.71 < outputs) & (outputs < 1.71))) |  # Regra para 0
		((targets_batch == 1) & (1.71 < outputs))     # Regra para 1
	)

print(matches.float().mean())
	
# Imprimir os resultados da previsão
for i, pred in enumerate(output):
	print(f"Resultado da previsão para o par {i+1}: {pred.item()}")

In [ ]:
model_order(sequences1_batch[0], sequences2_batch[0])

In [ ]:
sequences1 = embedding_generator.text_to_embedding_parallel(df_validation['content1'].tolist(), max_workers=20)


In [ ]:
from datasets import load_dataset
from scipy.stats import pearsonr, spearmanr

# Carregar o conjunto de dados de avaliação (STS Benchmark)
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Listas para armazenar as pontuações reais e as previsões do modelo
true_scores = []
pred_scores = []

with torch.no_grad():
    for example in eval_dataset:
        sent1 = example['sentence1']
        sent2 = example['sentence2']
        score = example['score']

        # Gerar embeddings para ambas as sentenças
        embedding1 = text_to_embedding(sent1).to('cuda')
        embedding2 = text_to_embedding(sent2).to('cuda')

        # Obter a previsão do modelo
        prediction = model_order([embedding1], [embedding2])
        pred_score = 1 / (1 + abs(prediction.item()))

        # Armazenar as pontuações
        true_scores.append(score)
        pred_scores.append(pred_score)

# Calcular as métricas de correlação
pearson_corr, _ = pearsonr(true_scores, pred_scores)
spearman_corr, _ = spearmanr(true_scores, pred_scores)

print("Similaridade Inversa da Diferença Absoluta")
print(f"Correlação de Pearson: {pearson_corr:.4f}")
print(f"Correlação de Spearman: {spearman_corr:.4f}")

In [ ]:
import torch
from datasets import load_dataset
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm  # Para acompanhar o progresso

# Carregar o conjunto de dados de avaliação (STS Benchmark)
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Definir o intervalo de valores de k a serem testados
k_values = np.arange(0.001, 100.0, 0.01)
best_k = None
best_pearson = -1  # Inicialização com um valor baixo
best_spearman = -1

# Pré-computar todas as predições para evitar recalcular múltiplas vezes
true_scores = []
predictions = []

with torch.no_grad():
    for example in tqdm(eval_dataset, desc="Processando exemplos"):
        sent1 = example['sentence1']
        sent2 = example['sentence2']
        score = example['score']

        # Gerar embeddings para ambas as sentenças
        embedding1 = text_to_embedding(sent1).to('cuda')
        embedding2 = text_to_embedding(sent2).to('cuda')

        # Obter a previsão do modelo
        prediction = model_order([embedding1], [embedding2])
        predictions.append(prediction.item())

        # Armazenar as pontuações reais
        true_scores.append(score)

true_scores = np.array(true_scores)
predictions = np.array(predictions)

# Iterar sobre os valores de k para encontrar o melhor
for k in tqdm(k_values, desc="Buscando o melhor k"):
    pred_scores = np.exp(-k * np.abs(predictions))
    
    pearson_corr, _ = pearsonr(true_scores, pred_scores)
    spearman_corr, _ = spearmanr(true_scores, pred_scores)
    
    # Verificar se este k é o melhor até agora
    if pearson_corr > best_pearson:
        best_pearson = pearson_corr
        best_spearman = spearman_corr
        best_k = k

print("Similaridade Baseada em Exponencial")
print(f"Melhor valor de k: {best_k}")
print(f"Correlação de Pearson: {best_pearson:.4f}")
print(f"Correlação de Spearman: {best_spearman:.4f}")

In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from scipy.stats import pearsonr, spearmanr

# Verificar se CUDA está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Carregar o modelo SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2").to(device)

# Carregar o conjunto de dados de avaliação (STS Benchmark)
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Listas para armazenar as pontuações reais e as previsões do modelo
true_scores = []
pred_scores = []

# Obter as sentenças e os escores do conjunto de dados
sentences1 = eval_dataset['sentence1']
sentences2 = eval_dataset['sentence2']
scores = eval_dataset['score']

# Processar uma frase por vez
for i in range(len(eval_dataset)):
    sentence1 = sentences1[i]
    sentence2 = sentences2[i]
    true_score = scores[i]

    # Gerar embeddings para ambas as sentenças individualmente
    embedding1 = model.encode(sentence1, convert_to_tensor=True, device=device, show_progress_bar=False)
    embedding2 = model.encode(sentence2, convert_to_tensor=True, device=device, show_progress_bar=False)

    # Calcular a similaridade cosseno
    cosine_score = util.cos_sim(embedding1, embedding2).item()

    # Escalar a similaridade cosseno de [-1, 1] para [0, 1]
    scaled_score = (cosine_score + 1) / 2

    # Armazenar as pontuações
    true_scores.append(true_score)
    pred_scores.append(scaled_score)

# Calcular as métricas de correlação
pearson_corr, _ = pearsonr(true_scores, pred_scores)
spearman_corr, _ = spearmanr(true_scores, pred_scores)

print(f"Correlação de Pearson: {pearson_corr:.4f}")
print(f"Correlação de Spearman: {spearman_corr:.4f}")

In [ ]:
import os
model_path = os.path.join('./saved_HuberLoss', 'regression_rnn_model.pth')

# Carregar o estado do modelo
#model_order.load_state_dict(torch.load(model_path, map_location='cuda'))

# Colocar o modelo em modo de avaliação
model_order.eval()

# Realizar a previsão
with torch.no_grad():
    output = model_order(
        [text_to_embedding(
            "O aluno estudou para o exame durante a noite.").to('cuda')],
        [text_to_embedding("O resultado foi excelente devido à dedicação.").to('cuda')])

# Imprimir a saída da regressão
print("Resultado da previsão:", output.item())

In [ ]:
del model_order  # Remove a referência ao modelo

# Libera a memória da GPU, se o modelo estiver usando CUDA
torch.cuda.empty_cache()

# Força a coleta de lixo
gc.collect()